# Scalar SurfacePDEs

## Laplace-Beltrami problem
We want to solve a problem of the form

$$
\begin{aligned}
          - \Delta_{\Gamma} u + u = & \, f 
          & & \text{on}~~ \Gamma,          
        \end{aligned}
$$

with $\Gamma := \{ \phi = 0 \}$. Here $\textbf{P} = \textbf{I} - \textbf{n}\textbf{n}^T$ is the projection onto the tangential space and

\begin{equation*}
\nabla_{\Gamma} u = \textbf{P} \nabla u, \qquad \operatorname{div}_{\Gamma}u = \operatorname{tr}(\nabla_{\Gamma}u)
\end{equation*}

are the surface gradient for scalar-valued functions and the surface divergence operator. Moreover, the Laplace-Beltrami operator is defined as follows:
\begin{equation*}
\Delta_{\Gamma} u = \operatorname{div}_{\Gamma} (\nabla_{\Gamma} u).
\end{equation*}

In [ ]:
# ngsolve stuff
from ngsolve import *
# visualization stuff
from ngsolve.internal import *
# basic xfem functionality
from xfem import *
# basic geometry features (for the background mesh)
from netgen.csg import CSGeometry, OrthoBrick, Pnt
# pi
from math import pi

# material parameter:
reac_coeff=1
diff_coeff=1

## Geometry approximation
We use a piecewise linear levelset approximation. 

First, we generate the background mesh of the domain and use a simplicial triangulation.

In [ ]:
cube = CSGeometry()
cube.Add (OrthoBrick(Pnt(-1.5,-1.5,-1.5), Pnt(1.5,1.5,1.5)))
mesh = Mesh (cube.GenerateMesh(maxh=0.2, quad_dominated=False))

On the background mesh we define the level set function and do a P1-interpolation.

In [ ]:
phi = sqrt(x**2+y**2+z**2)-1
lsetp1 = GridFunction(H1(mesh,order=1))
InterpolateToP1(phi,lsetp1)

## TraceFE / CutFE spaces
For the discretization we use standard background FESpaces restricted to the tetrahedrons cut by $\Gamma_h$:
$$
V_h^\Gamma = V_h |_{\Omega^{\Gamma}_h},
$$
with $\Omega^{\Gamma}_h = \cup \{ T \in \mathcal{T}_h \mid T \cap \Gamma_h \neq \emptyset \}$. In NGSolve we will simply take the space $V_h$ but mark the irrelevant dofs using the CutInfo-class:

In [ ]:
# FESpace 
VhG = H1(mesh, order=1)
u,v = VhG.TnT()

# overwrite freedofs of VhG to mark only dofs that are involved in the cut problem
ci = CutInfo(mesh, lsetp1)
freedofs = VhG.FreeDofs()
freedofs &= GetDofsOfElements(VhG,ci.GetElementsOfType(IF))

gfu = GridFunction(VhG)

Some helper coefficient functions:

In [ ]:
# normal vector
n = 1.0/grad(lsetp1).Norm() * grad(lsetp1)

# mesh size parameter
h = specialcf.mesh_size

# define tangential projection
def P(u):
   return u - (u*n)*n

# integration domains (and integration parameter "subdivlvl" and "force_intorder")
lset_if  = { "levelset" : lsetp1, "domain_type" : IF , "subdivlvl" : 0}

## TraceFEM / CutFEM discretization
Find $u_h \in  V_h^{\Gamma}$ such that
\begin{equation*}
\int_{\Gamma_h} \nabla_{\Gamma_h} u_h \cdot \nabla_{\Gamma_h} v_h \, ds_h + \int_{\Gamma_h} u_h v_h \, ds_h + \rho \int_{\Omega_{h}^{\Gamma}} (n_h \cdot \nabla u_h) (n_h \cdot \nabla v_h)  \, dx = \int_{\Gamma_h} f_h v_h \, ds_h ~~~ \text{for all}~ v_h \in V_h^{\Gamma}
\end{equation*}
with 
$$
  n_h = \nabla \phi_h^{lin}/\Vert \nabla \phi_h^{lin} \Vert.
$$

In [ ]:
# expressions of test and trial functions:
a = BilinearForm(VhG, symmetric = True)
a += SymbolicBFI(levelset_domain = lset_if, 
                 form = diff_coeff * P(grad(u)) * P(grad(v)) + reac_coeff * u * v)
a += SymbolicBFI(form = (diff_coeff/h+reac_coeff*h)*(grad(u)*n) * (grad(v)*n), 
                 definedonelements=ci.GetElementsOfType(IF))

f_coeff = (sin(pi*z)*(diff_coeff*pi*pi*(1-z*z)+reac_coeff)+diff_coeff*cos(pi*z)*2*pi*z)

f = LinearForm(VhG)
f += SymbolicLFI(levelset_domain = lset_if, form = f_coeff * v)


In [ ]:
a.Assemble()
f.Assemble()

We can now solve the problem (recall that freedofs only marks relevant dofs):

In [ ]:
gfu.vec[:] = 0.0
gfu.vec.data = a.mat.Inverse(freedofs) * f.vec

exact = sin(pi*z)
err_sqr_coefs = (gfu-exact)*(gfu-exact)
l2error = sqrt( Integrate( levelset_domain=lset_if, cf=err_sqr_coefs[0], mesh=mesh, order=2))
print ("l2error : ", l2error)

## Visualization on the active mesh


In [ ]:
from netgen import gui
%gui tk
Draw(gfu, mesh, "u")

well... not so nice.

## Visualization with VTK
Create vtk-output file to be read by Paraview.

In [ ]:
vtk = VTKOutput(ma=mesh,coefs=[lsetp1,gfu,exact],names=["P1-levelset","u","uSol"],filename="laplacebeltrami3d",subdivision=0)
vtk.Do()

In [ ]:
%%bash
ls *.vtk -l

In [ ]:
#%%bash
#paraview laplacebeltrami3d.vtk

<center>
![title](graphics/tracefem_scalar.png)
</center>